<h1> IMPORTING DATA

<h2>Broadway Gross df + Initial Transformation

In [85]:
# import Broadway Gross data set from Kaggle (scraped from Broadway League)
import pandas as pd
bway = pd.read_csv('data/KAGGLE_bway_grosses_1985_to_present.csv')

# Set week_ending column (designating last day in week represented) to datetime)
bway['week_ending'] = pd.to_datetime(bway['week_ending'])

# Add first_preview column 
first_preview = bway.groupby('show').min()['week_ending']

bway = pd.merge(bway, first_preview, on='show',how='inner')


In [86]:
# delineate what columns must be totals, and which must be averages (based on title and domain knowledge)

test_bway_totals = bway[['show','weekly_gross', 'potential_gross','performances','previews']]

test_bway_avgs = bway[['show', 'weekly_gross', 'avg_ticket_price', 'pct_capacity', 'top_ticket_price', 'seats_in_theatre']]

totals = test_bway_totals.groupby('show').sum().sort_values(by='weekly_gross', ascending=False)
averages = test_bway_avgs.groupby('show').mean().sort_values(by='avg_ticket_price')

totals = pd.merge(totals, first_preview, on='show', how='inner')

# joining total and average columns

broadway_data = pd.merge(totals, averages, on='show',how='inner')
broadway_data


,weekly_gross_x,potential_gross,performances,previews,week_ending,weekly_gross_y,avg_ticket_price,pct_capacity,top_ticket_price,seats_in_theatre
show,,,,,,,,,,
The Lion King,1.677802e+09,1.732523e+09,9284,33,1997-10-19,1.438938e+06,107.171209,0.976834,184.373605,1716.054031
Wicked,1.364249e+09,1.324600e+09,6817,25,2003-10-12,1.595613e+06,111.466912,0.971573,270.245902,1821.701754
The Phantom of the Opera,1.251493e+09,1.326206e+09,13350,16,1988-01-10,7.462687e+05,65.381449,0.897125,158.326744,1605.797853
Chicago,6.798825e+08,1.011836e+09,9674,25,1996-10-27,5.581958e+05,72.587726,0.827031,176.239729,1178.946634
The Book of Mormon,6.567838e+08,6.064189e+08,3730,28,2011-02-27,1.397412e+06,160.369298,1.023183,469.285714,1061.872340
...,...,...,...,...,...,...,...,...,...,...
Senator Joe,5.661000e+03,0.000000e+00,0,3,1989-01-08,5.661000e+03,12.040000,0.117400,NaN,1334.000000
Raffi,0.000000e+00,0.000000e+00,6,0,1993-04-11,0.000000e+00,0.000000,0.000000,NaN,1056.000000
Young Frankenstein,0.000000e+00,0.000000e+00,485,3,2007-10-14,0.000000e+00,0.000000,0.000000,272.461538,900.923077


In [87]:
# Rename "The Musical" title additions that complicate later merge 

broadway_data.rename(index= {'Matilda The Musical': 'Matilda',
                    'Billy Elliot: The Musical': 'Billy Elliot',
                    'School of Rock - The Musical': 'School of Rock',
                    'Moulin Rouge! The Musical!': 'Moulin Rouge!',
                    'A Bronx Tale The Musical': 'A Bronx Tale',
                    'Pretty Woman: The Musical': 'Pretty Woman',
                    'Shrek the Musical': 'Shrek',
                    'Ghost The Musical': 'Ghost',
                    'Bring It On: The Musical': 'Bring It On',
                    'A Christmas Story, The Musical': 'A Christmas Story',
                    'Dracula, the Musical': 'Dracula',
                    'Amelie, A New Musical':'Amélie',
                    'Cyrano - The Musical': 'Cyrano'}, inplace=True)

In [89]:
#reset index for column renaming 
broadway_data = broadway_data.reset_index()

In [90]:
# renaming columns in preparation for merge

columns = ['title', 'BWAY_total_weekly_gross', 'BWAY_total_potential_gross', 'BWAY_total_performances', 
           'BWAY_total_previews', 'BWAY_last_performance', 'BWAY_avg_weekly_gross',
           'BWAY_avg_ticket_price', 'BWAY_avg_capacity', 'BWAY_avg_top_ticket_price', 'BWAY_avg_seats_in_theatre']
           
broadway_data.columns = columns

<br>

<h2>IMDB DATA SET (Scraped and unscraped) + Merge w Broadway Data

<h3> At this point, I imported the master set of all movies in IMDB made public by IMDB and listed here as IMDB_movies

In [91]:
# importing IMDB Movie data set (not scraped, publicly available) and setting columns

IMDB_movies = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t')

IMDB_movies.columns = ['FILM_tconst', 'FILM_titleType', 'FILM_primaryTitle', 'title', 'FILM_isAdult', 
                       'FILM_startYear', 'FILM_endYear', 'FILM_runtimeMinutes', 'FILM_genres']



/Users/nick/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<h3> I now imported my two scraped data sets - scraped_musical_movies (a scrape of 10,000 films in the "Musical" category) and new_scrape (scraped data with non-musical titles that are still title matches to the movie dataset.

In [92]:
# Merging IMDB_movies (unscraped) with two separate sets of scraped data, based on 'title'
# This gives many more features to the titles that were scraped (10,000 "musical movies" and 500+ additional titles
# listed in other genres. The resulting list is, essentially, a full list of all movies on IMDB, with ~10,000 of
# them having additional features, scraped from the web 

scraped_musical_movies = pd.read_csv('data/IMDB_scraped_data.csv')
new_scrape = pd.read_csv('data/ORIGINAL.new_scrape.csv')

all_scrape = pd.concat([scraped_musical_movies, new_scrape])

merged_movies = pd.merge(all_scrape, IMDB_movies, on='title',how='outer')

In [93]:
len(merged_movies)

7330702

In [94]:
broadway_data

,title,BWAY_total_weekly_gross,BWAY_total_potential_gross,BWAY_total_performances,BWAY_total_previews,BWAY_last_performance,BWAY_avg_weekly_gross,BWAY_avg_ticket_price,BWAY_avg_capacity,BWAY_avg_top_ticket_price,BWAY_avg_seats_in_theatre
0,The Lion King,1.677802e+09,1.732523e+09,9284,33,1997-10-19,1.438938e+06,107.171209,0.976834,184.373605,1716.054031
1,Wicked,1.364249e+09,1.324600e+09,6817,25,2003-10-12,1.595613e+06,111.466912,0.971573,270.245902,1821.701754
2,The Phantom of the Opera,1.251493e+09,1.326206e+09,13350,16,1988-01-10,7.462687e+05,65.381449,0.897125,158.326744,1605.797853
3,Chicago,6.798825e+08,1.011836e+09,9674,25,1996-10-27,5.581958e+05,72.587726,0.827031,176.239729,1178.946634
4,The Book of Mormon,6.567838e+08,6.064189e+08,3730,28,2011-02-27,1.397412e+06,160.369298,1.023183,469.285714,1061.872340
...,...,...,...,...,...,...,...,...,...,...,...
1117,Senator Joe,5.661000e+03,0.000000e+00,0,3,1989-01-08,5.661000e+03,12.040000,0.117400,NaN,1334.000000
1118,Raffi,0.000000e+00,0.000000e+00,6,0,1993-04-11,0.000000e+00,0.000000,0.000000,NaN,1056.000000
1119,Young Frankenstein,0.000000e+00,0.000000e+00,485,3,2007-10-14,0.000000e+00,0.000000,0.000000,272.461538,900.923077
1120,Basia on Broadway,0.000000e+00,0.000000e+00,0,0,1994-11-20,0.000000e+00,0.000000,0.000000,NaN,1788.000000


In [107]:
# This merges the 7 million + entries on IMDB with broadway_data, creating a cross section of movies and musicals
# since 1985 that share titles (most reliably metric for determining an adaptation)

musical_movies = pd.merge(merged_movies, broadway_data, on='title', how='inner')

In [108]:
len(musical_movies)

99523

In [109]:
# Creates mask to get specifically movies from IMDB data set (no tv shows with title match, etc.)
isamovie = (musical_movies['FILM_titleType'] == 'movie') | (musical_movies['FILM_titleType'] == 'tvMovie')
musical_movies = musical_movies[isamovie]

<h3>Reset index from pairing down 7 million+ IMDB mobies to 27,084 individual movie entries (with MANY title duplicates), addressed later in the Notebook 

In [110]:
musical_movies = musical_movies.reset_index()
musical_movies

,index,Unnamed: 0,Unnamed: 0.1,title,runtime,stars,directors,filming_location,aka,oscars_won,...,BWAY_total_weekly_gross,BWAY_total_potential_gross,BWAY_total_performances,BWAY_total_previews,BWAY_last_performance,BWAY_avg_weekly_gross,BWAY_avg_ticket_price,BWAY_avg_capacity,BWAY_avg_top_ticket_price,BWAY_avg_seats_in_theatre
0,1,0.0,0.0,Beauty and the Beast,129,"['Emma Watson', 'Dan Stevens', 'Luke Evans']",['Bill Condon'],"London, England, UK",Beauty and the Beast,2.0,...,4.291585e+08,425324962.0,5461,46,1994-03-13,6.219688e+05,56.526377,0.860459,94.940096,1603.814493
1,2,0.0,0.0,Beauty and the Beast,129,"['Emma Watson', 'Dan Stevens', 'Luke Evans']",['Bill Condon'],"London, England, UK",Beauty and the Beast,2.0,...,4.291585e+08,425324962.0,5461,46,1994-03-13,6.219688e+05,56.526377,0.860459,94.940096,1603.814493
2,4,0.0,0.0,Beauty and the Beast,129,"['Emma Watson', 'Dan Stevens', 'Luke Evans']",['Bill Condon'],"London, England, UK",Beauty and the Beast,2.0,...,4.291585e+08,425324962.0,5461,46,1994-03-13,6.219688e+05,56.526377,0.860459,94.940096,1603.814493
3,5,0.0,0.0,Beauty and the Beast,129,"['Emma Watson', 'Dan Stevens', 'Luke Evans']",['Bill Condon'],"London, England, UK",Beauty and the Beast,2.0,...,4.291585e+08,425324962.0,5461,46,1994-03-13,6.219688e+05,56.526377,0.860459,94.940096,1603.814493
4,7,0.0,0.0,Beauty and the Beast,129,"['Emma Watson', 'Dan Stevens', 'Luke Evans']",['Bill Condon'],"London, England, UK",Beauty and the Beast,2.0,...,4.291585e+08,425324962.0,5461,46,1994-03-13,6.219688e+05,56.526377,0.860459,94.940096,1603.814493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27079,99511,NaN,NaN,The Tale of the Allergist's Wife,NaN,NaN,NaN,NaN,NaN,NaN,...,2.931073e+07,49362252.0,777,25,2000-10-15,2.902052e+05,47.757426,0.729550,71.534653,1046.000000
27080,99512,NaN,NaN,Next to Normal,NaN,NaN,NaN,NaN,NaN,NaN,...,3.176449e+07,58734916.0,734,20,2009-03-29,3.343630e+05,66.208842,0.813911,200.000000,772.000000
27081,99514,NaN,NaN,Spamalot,NaN,NaN,NaN,NaN,NaN,NaN,...,1.680674e+08,197496726.0,1575,34,2005-02-20,8.238596e+05,79.236078,0.885666,259.154412,1426.941176
27082,99516,NaN,NaN,Be More Chill,NaN,NaN,NaN,NaN,NaN,NaN,...,1.421779e+07,22790881.0,177,30,2019-02-17,5.468382e+05,92.808077,0.824885,253.730769,890.692308


<br>

<h1>FEATURE TRANSFORMATION

<h2>Column Re-naming

In [113]:
musical_movies.columns

Index(['title', 'runtime', 'stars', 'directors', 'filming_location', 'aka',
       'oscars_won', 'genres', 'budget', 'domestic_gross', 'worldwide_gross',
       'mpaa_rating', 'release', 'FILM_tconst', 'FILM_primaryTitle',
       'FILM_runtimeMinutes', 'FILM_genres', 'BWAY_total_weekly_gross',
       'BWAY_total_potential_gross', 'BWAY_total_performances',
       'BWAY_total_previews', 'BWAY_last_performance', 'BWAY_avg_weekly_gross',
       'BWAY_avg_ticket_price', 'BWAY_avg_capacity',
       'BWAY_avg_top_ticket_price', 'BWAY_avg_seats_in_theatre'],
      dtype='object')

In [115]:
# drops irrelevant columns and renames FILM and BWAY columns

musical_movies = musical_movies.drop(columns=['index', 'Unnamed: 0', 'Unnamed: 0.1', 'FILM_isAdult', 'FILM_titleType', 'FILM_endYear'])
musical_movies = musical_movies.drop(columns=['FILM_startYear'])
musical_movies.columns = ['title', 'FILM_runtime', 'FILM_stars', 'FILM_directors', 'FILM_filming_locations',
                         'FILM_aka', 'FILM_oscars_won', 'FILM_genres', 'FILM_budget', 'FILM_domestic_gross',
                         'FILM_worldwide_gross', 'FILM_mpaa_rating', 'FILM_release_date', 'FILM_tconst', 'FILM_primaryTitle', 
                        'FILM_runtimeMinutes', 'FILM_genres', 
                          'BWAY_total_weekly_gross', 'BWAY_total_potential_gross', 'BWAY_total_performances', 
                          'BWAY_total_previews', 'BWAY_last_performance', 'BWAY_avg_weekly_gross',
                          'BWAY_avg_ticket_price', 'BWAY_avg_capacity', 'BWAY_avg_top_ticket_price', 'BWAY_avg_seats_in_theatre']

<h2> Title

In [116]:
len(musical_movies.title)

27084

In [117]:
# determining number of unique titles

len(musical_movies.title.unique())

639

In [118]:
len(musical_movies.title.unique()) / len(musical_movies)

0.023593265396544084

<h4> Unique titles account for 639 of the 27,084 entries - 2.3%

<h2> Release Date

In [119]:
# Creating separate list for release_date, to be text transformed and eventually converted to datetime

release_date = musical_movies.FILM_release_date
release_date

0        17 March 2017 (USA)
1        17 March 2017 (USA)
2        17 March 2017 (USA)
3        17 March 2017 (USA)
4        17 March 2017 (USA)
                ...         
27079                    NaN
27080                    NaN
27081                    NaN
27082                    NaN
27083                    NaN
Name: FILM_release_date, Length: 27084, dtype: object

In [120]:
#transforming release date of movie 
for i,release in enumerate(release_date):
    #try:
    release_date[i] = str(release_date[i]).split('(')[0]
    release_date[i] = str(release_date[i]).strip()
    print(release_date[i])
    
release_date

/Users/nick/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-120-f798bc71798f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  release_date[i] = str(release_date[i]).strip()
<ipython-input-120-f798bc71798f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  release_date[i] = str(release_date[i]).

17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
17 March 2017
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
22 November 1991
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
April 1987
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 2018
15 November 20

12 May 1965
12 May 1965
12 May 1965
12 May 1965
12 May 1965
12 May 1965
12 May 1965
12 May 1965
12 May 1965
12 May 1965
12 May 1965
12 May 1965
12 May 1965
12 May 1965
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
13 March 2015
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 December 1958
26 D

2 May 2013
2 May 2013
2 May 2013
2 May 2013
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
19 January 1937
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
18 April 2016
18 April 201

2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
28 December 1914
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
4 March 1950
1963


...

Not found
Not found
Not found
Not found
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
24 January 1963
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
15 March 1973
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
September 2004
4 January 1956
4 January 1956
4 January 1956
4 January 1956
4 January 1956
4 Jan

16 May 2019
16 May 2019
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
1979
1979
1979
1979
1979
1979
1979
1979
1979
30 November 1977
30 November 1977
30 November 1977
16 January 2004
16 January 2004
16 January 2004
25 March 2018
25 March 2018
25 March 2018
16 December 1977
17 May 1978
17 May 1978
7 March 1957
7 March 1957
14 December 1978
14 December 1978
14 December 1978
14 December 1978
14 December 1978
14 December 1978
19 October 1990
19 October 1990
19 October 1990
19 October 1990
19 October 1990
19 October 1990
2 August 1996
2 August 1996
2 August 1996
2 August 1996
2 August 1996
2 August 1996
8 October 1979
1978
3 May 1979
13 June 1980
13 June 1980
13 June 1980
13 June 1980
13 June 1980
13 June 1980
18 May 1990
18 May 1990
18 May 1990
18 May 1990
18 May 1990
18 May 1990
26 October 2002
26 October 2002
26 October 2002
26 October 2002
26 October 2002
26 October 2002
5 December 2008
5 December 2008
5 December 2008
5 December 2008
5 December

12 April 2011
12 April 2011
12 April 2011
12 April 2011
12 April 2011
12 April 2011
12 April 2011
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
6 September 2013
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
August 2014
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
16 May 1997
17 January 1997
17 January 1997
17 January 1997
17 January 1997
17 January 1997
17 January 1997
17 January 1997
17 January 1997
17 January 1997
17 January 1997
17 October 2002
17 October 2002
17 October 2002
17 October 2002
17 

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


0        17 March 2017
1        17 March 2017
2        17 March 2017
3        17 March 2017
4        17 March 2017
             ...      
27079              nan
27080              nan
27081              nan
27082              nan
27083              nan
Name: FILM_release_date, Length: 27084, dtype: object

In [121]:
# Sets new release_date list to the release column 

musical_movies['FILM_release_date'] = release_date

In [122]:
# updating 9 to 5 and Ghost movie/musical combos 

musical_movies.loc[musical_movies.index == 593, 'FILM_release_year'] = 1980
musical_movies.loc[musical_movies.index == 594, 'FILM_release_year'] = 1990

In [123]:
# create release year column 
release_year = []
for i, date in enumerate(musical_movies['FILM_release_date']):
    release = musical_movies['FILM_release_date'][i].split(' ')[-1]
    release_year.append(release)
release_year


['2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1991',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '1987',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2020',
 '2020',
 '2020',
 '1978',
 '1978',
 '2019',
 '2019',
 '1994',
 '1994',
 '2021',
 '2021',
 '1961',
 '1961',
 '1961',
 '1961',
 '2021',
 '2021',
 '2005',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '1992',
 '1992',
 '1992',
 '1992',
 '1992',
 '1992',
 '1992',
 '1992',
 '1992',
 

In [124]:
# sets the release_year column to our new release_year list, creates numeric values, and converts to datetime

musical_movies['FILM_release_year'] = release_year
musical_movies['FILM_release_year'] = pd.to_numeric(musical_movies["FILM_release_year"], errors='coerce')

# fill numeric value with mean - there is a small amount of movies with no release_year successfully scraped, 
# so this seemed like the best way to handle these data values, in preparation for them being converted to datetime

musical_movies.FILM_release_year = musical_movies.FILM_release_year.fillna(musical_movies.FILM_release_year.mean())
musical_movies['FILM_release_year'] = pd.to_datetime(musical_movies['FILM_release_year'], format='%Y')


    

<h2> FILM - Gross (Domestic + Worldwide)

In [125]:
# Cleans text for string columns containing numeric values, converted to numeric values below 
    
for gross1 in musical_movies['FILM_domestic_gross']:
    musical_movies['FILM_domestic_gross'] = musical_movies['FILM_domestic_gross'].str.replace('[$,]', '')
    musical_movies['FILM_domestic_gross'] = musical_movies['FILM_domestic_gross'].str.strip()

for gross2 in musical_movies['FILM_worldwide_gross']:
    musical_movies['FILM_worldwide_gross'] = musical_movies['FILM_worldwide_gross'].str.replace('[$,]', '')
    musical_movies['FILM_worldwide_gross'] = musical_movies['FILM_worldwide_gross'].str.strip()


<h2>FILM - Budget

In [126]:
# appends a new budget_list, which will become our clean numeric budget feature 

budget_list = []

for i,budget in enumerate(musical_movies.FILM_budget):
    #print(i,budget)
    new_budget = musical_movies.FILM_budget[i]
    new_budget = str(new_budget).strip()
    new_budget = new_budget.replace(',', '')
    new_budget = new_budget.replace('$', '')
    budget_list.append(new_budget)

In [127]:
# assigns new budget_list to FILM_budget column 

musical_movies['FILM_budget'] = budget_list

<h2> FILM - MPAA Rating

In [128]:
# Creates numerical value correlated to a rating scale 
# G - 1 / PG - 2 / PG-13 - 3 / R - 4 / TV-MA - 5

rating_list = []

for i,rating in enumerate(musical_movies['FILM_mpaa_rating']):
    if musical_movies['FILM_mpaa_rating'][i] == 'TV-MA':
        rating_list.append('5')
    elif musical_movies['FILM_mpaa_rating'][i] == 'R':
        rating_list.append('4')
    elif musical_movies['FILM_mpaa_rating'][i] == 'PG-13':
        rating_list.append('3')
    elif musical_movies['FILM_mpaa_rating'][i] == 'PG':
        rating_list.append('2')
    elif musical_movies['FILM_mpaa_rating'][i] == 'TV-G':
        rating_list.append('1')
    elif musical_movies['FILM_mpaa_rating'][i] == 'G':
        rating_list.append('1')
    else:
        rating_list.append('0')

In [129]:
# Assigns rating_list created above to proper column in musical_movies table 

musical_movies['FILM_mpaa_rating'] = rating_list

<h2> FILM - Multi-Category - Numeric Conversion

In [130]:
# converts former strings to numeric values

musical_movies['FILM_runtime'] = pd.to_numeric(musical_movies["FILM_runtime"], errors='coerce')
musical_movies['FILM_budget'] = pd.to_numeric(musical_movies["FILM_budget"], errors='coerce')
musical_movies['FILM_domestic_gross'] = pd.to_numeric(musical_movies["FILM_domestic_gross"], errors='coerce')
musical_movies['FILM_worldwide_gross'] = pd.to_numeric(musical_movies["FILM_worldwide_gross"], errors='coerce')
musical_movies['FILM_mpaa_rating_number'] = pd.to_numeric(musical_movies['FILM_mpaa_rating'], errors='coerce')


In [143]:
musical_movies.to_csv('data/pre_MICE_backup')

In [34]:
musical_movies = pd.read_csv('data/pre_MICE_backup.csv')

In [35]:
musical_movies

,Unnamed: 0,title,FILM_runtime,FILM_stars,FILM_directors,FILM_filming_locations,FILM_aka,FILM_oscars_won,FILM_genres,FILM_budget,...,BWAY_total_performances,BWAY_total_previews,BWAY_last_performance,BWAY_avg_weekly_gross,BWAY_avg_ticket_price,BWAY_avg_capacity,BWAY_avg_top_ticket_price,BWAY_avg_seats_in_theatre,FILM_release_year,FILM_mpaa_rating_number
0,0,Beauty and the Beast,129.0,"['Emma Watson', 'Dan Stevens', 'Luke Evans']",['Bill Condon'],"London, England, UK",Beauty and the Beast,2.0,"['Family', 'Fantasy', 'Musical', 'Romance']",160000000.0,...,5461,46,1994-03-13,6.219688e+05,56.526377,0.860459,94.940096,1603.814493,2017-01-01,2
1,1,Beauty and the Beast,129.0,"['Emma Watson', 'Dan Stevens', 'Luke Evans']",['Bill Condon'],"London, England, UK",Beauty and the Beast,2.0,"['Family', 'Fantasy', 'Musical', 'Romance']",160000000.0,...,5461,46,1994-03-13,6.219688e+05,56.526377,0.860459,94.940096,1603.814493,2017-01-01,2
2,2,Beauty and the Beast,129.0,"['Emma Watson', 'Dan Stevens', 'Luke Evans']",['Bill Condon'],"London, England, UK",Beauty and the Beast,2.0,"['Family', 'Fantasy', 'Musical', 'Romance']",160000000.0,...,5461,46,1994-03-13,6.219688e+05,56.526377,0.860459,94.940096,1603.814493,2017-01-01,2
3,3,Beauty and the Beast,129.0,"['Emma Watson', 'Dan Stevens', 'Luke Evans']",['Bill Condon'],"London, England, UK",Beauty and the Beast,2.0,"['Family', 'Fantasy', 'Musical', 'Romance']",160000000.0,...,5461,46,1994-03-13,6.219688e+05,56.526377,0.860459,94.940096,1603.814493,2017-01-01,2
4,4,Beauty and the Beast,129.0,"['Emma Watson', 'Dan Stevens', 'Luke Evans']",['Bill Condon'],"London, England, UK",Beauty and the Beast,2.0,"['Family', 'Fantasy', 'Musical', 'Romance']",160000000.0,...,5461,46,1994-03-13,6.219688e+05,56.526377,0.860459,94.940096,1603.814493,2017-01-01,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27079,27079,The Tale of the Allergist's Wife,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,777,25,2000-10-15,2.902052e+05,47.757426,0.729550,71.534653,1046.000000,1984-01-01,0
27080,27080,Next to Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,734,20,2009-03-29,3.343630e+05,66.208842,0.813911,200.000000,772.000000,1984-01-01,0
27081,27081,Spamalot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1575,34,2005-02-20,8.238596e+05,79.236078,0.885666,259.154412,1426.941176,1984-01-01,0
27082,27082,Be More Chill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,177,30,2019-02-17,5.468382e+05,92.808077,0.824885,253.730769,890.692308,1984-01-01,0


<h2>NaN values

<h3> Due to web scraping and a lack of full information (budget and domestic gross) available for many of these movies, NaN values were a significant issue. <br><br>After research on handling NaN values and consulting with fellow students, I decided to use MICE to impute the most effective NaN values to accomodate for error from the scrape.

In [36]:
import fancyimpute  
from fancyimpute import IterativeImputer

from sklearn.impute import SimpleImputer 
# Getting only the float columns
continuous = musical_movies[['FILM_budget', 'FILM_oscars_won', 'FILM_domestic_gross', 'FILM_worldwide_gross',
                             'FILM_mpaa_rating_number', 'FILM_runtime', 'BWAY_total_weekly_gross', 'BWAY_total_potential_gross']]
MICE = IterativeImputer()
continuous = MICE.fit_transform(continuous)
continuous = pd.DataFrame(continuous, columns = ['FILM_budget', 'FILM_oscars_won', 
                                          'FILM_domestic_gross', 'FILM_worldwide_gross',
                                          'FILM_mpaa_rating_number', 
                                          'FILM_runtime', 'BWAY_total_weekly_gross', 'BWAY_total_potential_gross']).set_index(musical_movies.index)
continuous

/Users/nick/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:669: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


,FILM_budget,FILM_oscars_won,FILM_domestic_gross,FILM_worldwide_gross,FILM_mpaa_rating_number,FILM_runtime,BWAY_total_weekly_gross,BWAY_total_potential_gross
0,1.600000e+08,2.000000,5.044812e+08,1.264064e+09,2.0,129.000000,4.291585e+08,425324962.0
1,1.600000e+08,2.000000,5.044812e+08,1.264064e+09,2.0,129.000000,4.291585e+08,425324962.0
2,1.600000e+08,2.000000,5.044812e+08,1.264064e+09,2.0,129.000000,4.291585e+08,425324962.0
3,1.600000e+08,2.000000,5.044812e+08,1.264064e+09,2.0,129.000000,4.291585e+08,425324962.0
4,1.600000e+08,2.000000,5.044812e+08,1.264064e+09,2.0,129.000000,4.291585e+08,425324962.0
...,...,...,...,...,...,...,...,...
27079,1.521161e+07,0.084864,4.121988e+07,7.182692e+07,0.0,102.980430,2.931073e+07,49362252.0
27080,1.501464e+07,0.094996,4.367441e+07,7.423149e+07,0.0,102.380877,3.176449e+07,58734916.0
27081,1.778052e+07,0.108597,6.602247e+07,1.305057e+08,0.0,99.480063,1.680674e+08,197496726.0
27082,1.530758e+07,0.067242,3.539999e+07,6.332737e+07,0.0,104.193830,1.421779e+07,22790881.0


In [38]:
# assign columns from MICE imputer to original df

column_list = ['FILM_budget', 'FILM_oscars_won', 'FILM_domestic_gross', 'FILM_worldwide_gross','FILM_mpaa_rating_number', 
 'FILM_runtime', 'BWAY_total_weekly_gross', 'BWAY_total_potential_gross']

for column in column_list:
    musical_movies[column] = continuous[column]
    
    

<h2>Dropping Values based on highest-grossing title

<h3>Drops all duplicates, leaving only the highest-grossing title (for 6 Beauty and the Beasts, only the highest grossing film [2018 remake] is considered)

In [39]:
musical_movies = musical_movies.groupby('title', group_keys=False).apply(lambda x: x.loc[x.FILM_domestic_gross.idxmax()])

musical_movies



,Unnamed: 0,title,FILM_runtime,FILM_stars,FILM_directors,FILM_filming_locations,FILM_aka,FILM_oscars_won,FILM_genres,FILM_budget,...,BWAY_total_performances,BWAY_total_previews,BWAY_last_performance,BWAY_avg_weekly_gross,BWAY_avg_ticket_price,BWAY_avg_capacity,BWAY_avg_top_ticket_price,BWAY_avg_seats_in_theatre,FILM_release_year,FILM_mpaa_rating_number
title,,,,,,,,,,,,,,,,,,,,,
"'night, Mother",24792,"'night, Mother",96.000000,"['Sissy Spacek', 'Anne Bancroft', 'Ed Berke']",['Tom Moore'],"Universal Studios - 100 Universal City Plaza, ...","'night, Mother",1.0,"['contentRating: PG-13', 'actor: [', '{', '@ty...",5.237457e+06,...,65,26,2004-10-24,167623.333333,52.470000,0.399467,86.250000,1078.000000,1987-01-01,3.0
13,26229,13,104.039114,"['Ismo Kallio', 'Seija Lampila', 'Lasse Liemola']",['Aarre Elo'],Not found,Not found,0.0,Not found,1.511529e+07,...,105,22,2008-09-21,277273.062500,58.883125,0.548763,198.500000,1042.000000,1966-01-01,0.0
1776,5237,1776,141.000000,"['William Daniels', 'Howard Da Silva', 'Ken Ho...",['Peter H. Hunt'],Columbia/Warner Bros. Ranch - 411 North Hollyw...,1776,1.0,"['Drama', 'Family', 'History', 'Musical']",4.000000e+06,...,333,34,1997-07-20,297125.847826,40.464565,0.791802,72.065217,1351.173913,1972-01-01,1.0
1984,21055,1984,90.000000,"[""Edmond O'Brien"", 'Michael Redgrave', 'Jan St...",['Michael Anderson'],UK,1984,0.0,"['Drama', 'Sci-Fi']",1.513921e+07,...,125,38,2017-05-21,326658.100952,61.161905,0.713886,249.000000,970.000000,1956-01-01,0.0
42nd Street,5208,42nd Street,154.000000,"['Jamie Ross', 'Elizabeth Allen', 'Clare Leach']",['Mark Bramble'],"NHK Theatre, 2 Chome-2-1 Jinnan, Shibuya City,...",Not found,0.0,"['actor: [', '{', '@type: Person', 'url: /name...",1.000000e+00,...,3028,31,1985-06-09,464200.460938,45.022109,0.703124,99.016582,1720.968750,2020-01-01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wonderland,25913,Wonderland,120.000000,Not found,['Robert Oey'],Not found,Not found,0.0,"['director: {', '@type: Person', 'url: /name/n...",1.517104e+07,...,33,30,2011-03-27,554871.375000,52.387500,0.834075,200.000000,1612.000000,2004-01-01,0.0
Xanadu,5143,Xanadu,96.000000,"['Olivia Newton-John', 'Gene Kelly', 'Michael ...",['Robert Greenwald'],Pan-Pacific Auditorium - 7600 W. Beverly Boule...,Xanadu,0.0,"['Fantasy', 'Musical', 'Romance']",2.000000e+07,...,512,49,2007-05-27,241215.633803,70.229718,0.721730,226.250000,595.943662,1980-01-01,2.0
You Never Can Tell,18920,You Never Can Tell,120.000000,"['Michael Allaby', 'Doreen Aris', 'Alan Bridges']",Not found,"Birmingham, West Midlands, England, UK",Not found,0.0,"['Comedy', 'Drama']",1.538315e+07,...,125,23,1986-09-21,60722.000000,18.970000,0.613189,NaN,682.000000,1955-01-01,0.0


In [40]:
# Reseting index after massive cut
musical_movies = musical_movies.reset_index(drop=True)

In [47]:
# This dataframe reflects the values if we had not used MICE, but just replaced with the mean. 

df = pd.read_csv('data/pre_MICE_backup.csv')
sample = df['FILM_domestic_gross'].fillna(df['FILM_domestic_gross'].mean())
sample.describe()

count    2.708400e+04
mean     6.665767e+07
std      2.629414e+07
min      2.695000e+03
25%      6.665767e+07
50%      6.665767e+07
75%      6.665767e+07
max      6.593639e+08
Name: FILM_domestic_gross, dtype: float64

In [46]:
# This dataframe refelects the values with MICE replacement - a larger distribution that does not have 50%+ of the 
# dataset values reflecting the meme. 

musical_movies['FILM_domestic_gross'].describe()

count    6.390000e+02
mean     4.233849e+07
std      5.214296e+07
min      2.841000e+03
25%      3.042278e+07
50%      3.218099e+07
75%      3.563800e+07
max      6.593639e+08
Name: FILM_domestic_gross, dtype: float64

<h3>MICE was chosen as the Fill NaN solution 

<h2>Genre dummies

<h3>There were some unsuccessfully scraped genre categories (note index 4 below), but I now wanted to create dummy variables for genre of each movie.

In [48]:
# Cleans Genre category, and separates successfully scraped genres into list

import numpy as np

new_genre_list = []

for n, genre in enumerate(musical_movies.FILM_genres):
    if n == 0:
        genre_list = musical_movies.FILM_genres[n]
        genre_list = str(genre_list).replace("'", '')
        genre_list = genre_list.replace("]", '')
        genre_list = genre_list.replace("[", '')
        genre_list = genre_list.split(',')
        for i, subgenre in enumerate(genre_list):
            genre_list[i] = genre_list[i].strip()
            
        new_genre_list.append(genre_list)

        #real_data.xs(0)['FILM_genres'] = genre_list 

    else:
        genre_list = musical_movies.FILM_genres[n-1]
        genre_list = str(genre_list).replace("'", '')
        genre_list = genre_list.replace("]", '')
        genre_list = genre_list.replace("[", '')
        genre_list = genre_list.split(',')
        for i, subgenre in enumerate(genre_list):
            genre_list[i] = genre_list[i].strip()
            
        new_genre_list.append(genre_list)
        #real_data.xs(n-1)['FILM_genres'] = genre_list

In [49]:
musical_movies['FILM_genres'] = new_genre_list

In [50]:
# Backup code, incase FILM_genres has been saved as two separate columns
try:
    musical_movies = musical_movies.drop('FILM_genres')
except:
    pass

In [51]:
# Divides genres into five lists (only first five genres accepted)

musical_movies['GENRE_1'] = ''
musical_movies['GENRE_2'] = ''
musical_movies['GENRE_3'] = ''
musical_movies['GENRE_4'] = ''
musical_movies['GENRE_5'] = ''

genre_options = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
                'Family', 'Fantasy', 'Film Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance',
                'Sci-Fi', 'Short', 'Sport', 'Superhero', 'Thriller', 'War', 'Western']

# Specifics the entire selection of lists of genres
for n,genre_list in enumerate(musical_movies.FILM_genres):
    
    # Zooms in on the individual genre in each list
    for i, genre in enumerate(genre_list):
        
        # Qualifies if that listed genre is in the IMDB list of genres
        if genre in genre_options:

            if genre == genre_list[0]:
                musical_movies.loc[n,'GENRE_1']=genre
            elif genre == genre_list[1]:
                musical_movies.loc[n,'GENRE_2']=genre
            elif genre == genre_list[2]:
                musical_movies.loc[n,'GENRE_3']=genre
            elif genre == genre_list[3]:
                musical_movies.loc[n,'GENRE_4']=genre
            elif genre == genre_list[4]:
                musical_movies.loc[n,'GENRE_5']=genre
            else:
                pass

In [55]:
# There was no functionally effecient way to do this - this adds the individual dummies for each of the five
# genre categories, and gives a clean dummy variable from these five genres.

dummies = pd.get_dummies(musical_movies[['GENRE_1','GENRE_2', 'GENRE_3','GENRE_4','GENRE_5']])

musical_movies['GENRE_Action'] = dummies.GENRE_1_Action
musical_movies['GENRE_Adventure'] = dummies.GENRE_1_Adventure + dummies.GENRE_2_Adventure
musical_movies['GENRE_Adventure'] = dummies.GENRE_1_Adventure + dummies.GENRE_2_Adventure + dummies.GENRE_3_Adventure
musical_movies['GENRE_Animation'] = dummies.GENRE_1_Animation
musical_movies['GENRE_Biography'] = dummies.GENRE_1_Biography
musical_movies['GENRE_Comedy'] = dummies.GENRE_1_Comedy + dummies.GENRE_2_Comedy + dummies.GENRE_3_Comedy
musical_movies['GENRE_Crime'] = dummies.GENRE_1_Crime + dummies.GENRE_2_Crime + dummies.GENRE_3_Crime
musical_movies['GENRE_Documentary'] = dummies.GENRE_1_Documentary
musical_movies['GENRE_Drama'] = dummies.GENRE_1_Drama + dummies.GENRE_2_Drama + dummies.GENRE_3_Drama + dummies.GENRE_4_Drama
musical_movies['GENRE_Family'] = dummies.GENRE_1_Family + dummies.GENRE_2_Family + dummies.GENRE_3_Family + dummies.GENRE_4_Family
musical_movies['GENRE_Fantasy'] = dummies.GENRE_1_Fantasy + dummies.GENRE_2_Fantasy + dummies.GENRE_3_Fantasy + dummies.GENRE_5_Fantasy
musical_movies['GENRE_Horror'] = dummies.GENRE_1_Horror + dummies.GENRE_2_Horror
musical_movies['GENRE_Music'] = dummies.GENRE_2_Music + dummies.GENRE_3_Music + dummies.GENRE_4_Music
musical_movies['GENRE_Music'] = dummies.GENRE_1_Music + dummies.GENRE_2_Music + dummies.GENRE_3_Music + dummies.GENRE_4_Music
musical_movies['GENRE_Musical'] = dummies.GENRE_1_Musical + dummies.GENRE_2_Musical + dummies.GENRE_3_Musical + dummies.GENRE_4_Musical + dummies.GENRE_5_Musical
musical_movies['GENRE_Mystery'] = dummies.GENRE_2_Mystery + dummies.GENRE_3_Mystery
musical_movies['GENRE_Mystery'] = dummies.GENRE_1_Mystery + dummies.GENRE_2_Mystery + dummies.GENRE_3_Mystery + dummies.GENRE_4_Mystery
musical_movies['GENRE_Romance'] = dummies.GENRE_2_Romance + dummies.GENRE_3_Romance + dummies.GENRE_4_Romance + dummies.GENRE_5_Romance 
#musical_movies['GENRE_Sci-Fi'] = dummies.GENRE_2_Sci-Fi + dummies.GENRE_3_Sci-Fi + dummies.GENRE_4_Sci-Fi + dummies.GENRE_5_Sci-Fi
musical_movies['GENRE_Sport'] = dummies.GENRE_2_Sport + dummies.GENRE_3_Sport
musical_movies['GENRE_Thriller'] = dummies.GENRE_2_Thriller + dummies.GENRE_3_Thriller + dummies.GENRE_4_Thriller + dummies.GENRE_5_Thriller
musical_movies['GENRE_War'] = dummies.GENRE_2_War + dummies.GENRE_3_War + dummies.GENRE_5_War
musical_movies['GENRE_War'] = dummies.GENRE_2_War + dummies.GENRE_3_War + dummies.GENRE_4_War + dummies.GENRE_5_War
musical_movies['GENRE_Western'] = dummies.GENRE_3_Western 
musical_movies['GENRE_Western'] = dummies.GENRE_3_Western + dummies.GENRE_5_Western



In [56]:
# Reseting index after massive cut
real_data = real_data.reset_index(drop=True)

In [57]:
real_data.to_csv('data/final_dataset_10920.csv')

<h2> Housekeeping - dropping faulty values / columns

In [58]:
# dropped filming location, which appears irrelevant
musical_movies = musical_movies.drop(columns=['FILM_filming_locations'])

In [59]:
musical_movies.iloc[26]

Unnamed: 0                                                                22493
title                                                         A Thousand Clowns
FILM_runtime                                                                118
FILM_stars                    ['Jason Robards', 'Barbara Harris', 'Martin Ba...
FILM_directors                                                     ['Fred Coe']
FILM_aka                                                      A Thousand Clowns
FILM_oscars_won                                                               0
FILM_genres                                      [Drama, History, Romance, War]
FILM_budget                                                         1.80767e+06
FILM_domestic_gross                                                   5.232e+06
FILM_worldwide_gross                                               -2.04722e+07
FILM_mpaa_rating                                                              0
FILM_release_date                       

In [60]:
# Drops list of movies currently in production (faulty data), and resets index 

musical_movies = musical_movies.drop([49, 80, 157, 195, 201, 204, 309, 337, 382, 444, 454, 553, 629, 638])
musical_movies = musical_movies.reset_index(drop=True)

In [61]:
# Second drop of more irrelevant values...

musical_movies = musical_movies.drop([26, 87, 114, 245, 277, 345, 379, 551, 612])
musical_movies = musical_movies.reset_index(drop=True)

In [62]:
# Remove negative values for budget and worldwide gross (from MICE estimation)

musical_movies.loc[musical_movies.FILM_budget < 0] = 0
musical_movies.loc[musical_movies.FILM_worldwide_gross < 0] = 0

In [63]:
# test of abovea
musical_movies['FILM_worldwide_gross'].min()

398.0

In [65]:
musical_movies.to_csv('data/final_dataset_10920.csv')

<h2>Post-modeling feature engineering

<h3> Creation of BWAY_average_weekly_gross - incorporated into second model attempt

In [23]:
# BWAY_average_weekly_gross represents how much a show makes, on average, per week. 

avg_weekly_gross = musical_movies['BWAY_total_performances'] + musical_movies['BWAY_total_previews']

musical_movies['BWAY_average_weekly_gross'] = musical_movies.BWAY_total_weekly_gross / avg_weekly_gross


In [24]:
#test[test.BWAY_average_weekly_gross == test.BWAY_average_weekly_gross.max()]
musical_movies.BWAY_average_weekly_gross.describe()

count     27084.000000
mean      57955.158128
std       33253.508238
min           0.000000
25%       34131.430168
50%       51739.983333
75%       66869.158188
max      335064.961968
Name: BWAY_average_weekly_gross, dtype: float64

<h3>Creation of Gross Ratio, an ultimately negligible variable, due to the inefficacy of the "total_potential_gross" variable

In [319]:
for i, show in enumerate(musical_movies['BWAY_total_potential_gross']):
    performances_and_previews = musical_movies['BWAY_total_performances'][i] + musical_movies['BWAY_total_previews'][i]
    if musical_movies['BWAY_total_potential_gross'][i] == 0:
        musical_movies['BWAY_total_potential_gross'][i] = musical_movies['BWAY_avg_ticket_price'][i] * performances_and_previews
    else:
        pass
        

,Unnamed: 0,title,FILM_runtime,FILM_stars,FILM_directors,FILM_aka,FILM_oscars_won,FILM_genres,FILM_budget,FILM_domestic_gross,...,GENRE_Horror,GENRE_Music,GENRE_Musical,GENRE_Mystery,GENRE_Romance,GENRE_Sport,GENRE_Thriller,GENRE_War,GENRE_Western,BWAY_gross_ratio
0,0,"'night, Mother",96.000000,"['Sissy Spacek', 'Anne Bancroft', 'Ed Berke']",['Tom Moore'],"'night, Mother",1.0,},5.237457e+06,4.418630e+05,...,0,0,0,0,0,0,0,0,0,0.274451
1,1,13,104.039114,"['Ismo Kallio', 'Seija Lampila', 'Lasse Liemola']",['Aarre Elo'],Not found,0.0,Not found,1.511529e+07,3.374464e+07,...,0,0,0,0,0,0,0,0,0,0.350419
2,2,1776,141.000000,"['William Daniels', 'Howard Da Silva', 'Ken Ho...",['Peter H. Hunt'],1776,1.0,"['Drama', 'Family', 'History', 'Musical']",4.000000e+06,2.628552e+07,...,0,0,1,0,0,0,0,0,0,0.467476
3,3,1984,90.000000,"[""Edmond O'Brien"", 'Michael Redgrave', 'Jan St...",['Michael Anderson'],1984,0.0,"['Drama', 'Sci-Fi']",1.513921e+07,3.435207e+07,...,0,0,0,0,0,0,0,0,0,0.433314
4,4,42nd Street,154.000000,"['Jamie Ross', 'Elizabeth Allen', 'Clare Leach']",['Mark Bramble'],Not found,0.0,"['actor:', '{', '@type: Person', 'url: /name/n...",1.000000e+00,5.038395e+07,...,0,0,0,0,0,0,0,0,0,0.872423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,620,Wonderful Town,120.000000,"['Rosalind Russell', 'Sydney Chaplin', 'Jacque...","['Mel Ferber', 'Herbert Ross']",Not found,0.0,"['actor:', '{', '@type: Person', 'url: /name/n...",1.482855e+07,4.457849e+07,...,0,0,0,0,0,0,0,0,0,0.490172
612,621,Wonderland,120.000000,Not found,['Robert Oey'],Not found,0.0,"['director: {', '@type: Person', 'url: /name/n...",1.517104e+07,3.328188e+07,...,0,0,0,0,0,0,0,0,0,0.399504
613,622,Xanadu,96.000000,"['Olivia Newton-John', 'Gene Kelly', 'Michael ...",['Robert Greenwald'],Xanadu,0.0,"['Fantasy', 'Musical', 'Romance']",2.000000e+07,2.276257e+07,...,0,0,1,0,1,0,0,0,0,0.487504
614,623,You Never Can Tell,120.000000,"['Michael Allaby', 'Doreen Aris', 'Alan Bridges']",Not found,Not found,0.0,"['Comedy', 'Drama']",1.538315e+07,3.042472e+07,...,0,0,0,0,0,0,0,0,0,410.932625


In [304]:
# Creating gross ratio to determine relative success 
musical_movies['BWAY_gross_ratio'] = musical_movies.BWAY_total_weekly_gross / musical_movies.BWAY_total_potential_gross


In [316]:
musical_movies['BWAY_gross_ratio'].sort_values(ascending=False).sample(100)

271      0.544274
137      0.704911
43       0.463532
606      0.829945
441      0.550571
          ...    
526      0.411997
322    809.625194
61       0.693589
427      0.217156
561      0.491635
Name: BWAY_gross_ratio, Length: 100, dtype: float64

In [32]:
musical_movies.loc[442]

Unnamed: 0                               442
Unnamed: 0.1                             496
Unnamed: 0.1.1                           503
title                            The Heiress
FILM_runtime                             106
                                    ...     
FILM_budget_adj                      59558.6
FILM_domestic_gross_adj               123070
FILM_worldwide_gross_adj              237115
BWAY_total_weekly_gross_adj           127304
BWAY_average_weekly_gross_adj        246.237
Name: 442, Length: 62, dtype: object

In [331]:
musical_movies.to_csv('data/ORIGINAL_musical_movies_10820.csv')

<h3> CPI Integration - Consumer Price Index (for inflation, etc)

In [183]:
import pandas as pd

musical_movies = pd.read_csv('data/final_dataset_10920.csv')

In [186]:
# Import CPI
full_cpi = pd.read_csv('data/Scraped_Data/DATAHUB_cpi.csv')

In [187]:
# Convert both date columns to datetime
full_cpi['Date'] = pd.to_datetime(full_cpi['Date'])

In [188]:
# Create full CPI column, and drop unnecessary columns
# full_cpi ['cpi'] = full_cpi['cpi'].fillna(full_cpi['Index'])
full_cpi = full_cpi.drop(columns=['Inflation'])
full_cpi.columns = ['Date', 'cpi']

In [189]:
full_cpi

,Date,cpi
0,1913-01-01,9.800
1,1913-02-01,9.800
2,1913-03-01,9.800
3,1913-04-01,9.800
4,1913-05-01,9.700
...,...,...
1288,2020-05-01,256.493
1289,2020-06-01,257.797
1290,2020-07-01,259.101
1291,2020-08-01,259.918


<h2>DATETIME ETC 

In [190]:
# Further drop of irrelevant matches, or where there is not enough movie data available to be relevant. 


musical_movies = musical_movies.drop([23, 51,  60,  63,  94, 148, 176, 184, 189, 192, 230,
            259, 297, 320, 333, 334, 353, 357, 411, 454, 459, 476,
            481, 503, 547, 557, 571])

musical_movies = musical_movies.drop([20, 42, 57, 128, 140, 170, 219, 226, 247, 287, 290, 293, 294, 309, 377, 407, 417, 423, 431, 
                 473, 528, 566, 573, 593, 600, 604])

musical_movies = musical_movies.reset_index(drop=True)

musical_movies = musical_movies.drop([4, 31, 66, 96, 165, 273, 400, 415, 452, 456, 552, 553])

musical_movies = musical_movies.reset_index(drop=True)

musical_movies = musical_movies.drop([164, 413, 449])

musical_movies = musical_movies.reset_index(drop=True)



In [191]:
musical_movies['FILM_release_date'].unique()

array(['29 January 1987', '18 January 1966', '17 November 1972',
       'September 1956', nan, '1 October 1993', '15 May 2015',
       '6 November 2009', '2 January 2015', '13 March 2002', 'April 1976',
       '2012', '11 December 1992', '16 October 1966', '3 October 1979',
       '21 May 1980', '21 December 1988', '23 April 2020',
       '14 April 1985', '27 May 1975', '25 February 2008',
       '29 October 1995', '2 December 1980', '24 July 1996',
       '24 April 1974', '5 September 1996', '1988', '19 April 1940',
       'Not found', '23 August 1935', '26 December 1963',
       '10 September 1921', '10 December 1974', '28 April 1959',
       '8 October 1954', '26 August 1955', '24 May 2019', '20 April 2007',
       '12 March 2011', '19 September 1984', '13 September 1996',
       '19 August 2020', '19 January 2008', '20 September 2012',
       '11 November 1951', '2 December 1966', '5 June 2018', '4 May 1948',
       '21 November 1997', '8 May 1964', '1963', '19 December 2014',
    

In [192]:
musical_movies = musical_movies.dropna(subset=['FILM_release_date'])
musical_movies = musical_movies.reset_index(drop=True)

In [193]:
len(musical_movies)

544

In [194]:
musical_movies = musical_movies[musical_movies.FILM_release_date != 'Not found']
musical_movies = musical_movies.reset_index(drop=True)


In [195]:
len(musical_movies)

521

In [196]:
# Apply timedate to release date 

from datetime import datetime

release_list = []
fail = []

for i, release in enumerate(musical_movies['FILM_release_date']):
    try:
        s = str(musical_movies['FILM_release_date'][i])
        d = datetime.strptime(s, '%d %B %Y')
        d = pd.to_datetime(d)
        release_list.append(d)
    except ValueError:
        try:
            s = str(musical_movies['FILM_release_date'][i])
            d = datetime.strptime(s, '%B %Y')
            d = pd.to_datetime(d)
            release_list.append(d)
        except ValueError:
            try:
                s = str(musical_movies['FILM_release_date'][i])
                d = datetime.strptime(s, '%Y')
                d = pd.to_datetime(d)
                release_list.append(d)
            except:
                fail.append(musical_movies['title'][i])
    

In [197]:
# Confirm that all values made it to the release_list...

len(musical_movies['FILM_release_date']) - len(release_list)

0

In [198]:
# and set release date column to the release values 

musical_movies['FILM_release_date'] = release_list

In [199]:
# Set all film release dates to datetime
musical_movies['FILM_release_date'] = pd.to_datetime(musical_movies['FILM_release_date'])

In [200]:
# Create list of CPI relative to each movie release date 

test = musical_movies

cpi_list = []
fail_list = []

for n, release in enumerate(test['FILM_release_date']):
    #print(n)
    #print(release)
    for i, cpi in enumerate(full_cpi.Date):
        if full_cpi['Date'][i] >= test['FILM_release_date'][n]:
            cpi_list.append(full_cpi['cpi'][i])
            break
        elif full_cpi['Date'][i] > test['FILM_release_date'][n]:
            break
                
    #    print(i)
    #    print(full_cpi.cpi[i])

cpi_list

[111.6,
 32.0,
 42.5,
 27.4,
 145.7,
 242.06400000000002,
 215.949,
 240.172,
 179.8,
 56.1,
 226.665,
 142.6,
 32.9,
 75.9,
 82.7,
 121.1,
 256.493,
 107.3,
 53.6,
 213.528,
 153.6,
 87.0,
 157.3,
 48.6,
 158.3,
 115.7,
 14.0,
 13.7,
 30.9,
 17.5,
 52.1,
 29.0,
 26.8,
 26.9,
 262.925,
 207.949,
 224.90599999999998,
 105.3,
 158.3,
 259.918,
 211.69299999999998,
 231.317,
 26.5,
 32.9,
 257.902,
 24.1,
 161.3,
 31.0,
 30.4,
 239.81099999999998,
 23.8,
 26.9,
 211.08,
 13.6,
 21.5,
 36.4,
 166.2,
 109.7,
 185.2,
 248.37099999999998,
 236.185,
 236.185,
 94.9,
 231.407,
 251.12900000000002,
 117.1,
 29.6,
 250.519,
 118.5,
 186.2,
 198.7,
 117.1,
 47.2,
 233.59599999999998,
 24.1,
 250.987,
 30.1,
 180.1,
 27.5,
 30.0,
 160.0,
 51.9,
 216.33,
 111.2,
 173.7,
 45.9,
 139.5,
 244.547,
 233.877,
 151.4,
 246.467,
 252.562,
 56.1,
 177.4,
 142.6,
 33.8,
 137.8,
 26.8,
 233.06900000000002,
 105.0,
 181.7,
 164.0,
 226.889,
 196.4,
 203.9,
 110.4,
 29.0,
 174.0,
 35.6,
 241.34599999999998,
 16

In [201]:
# KEEP - test of CPI effectiveness
len(test) - len(cpi_list)

0

In [202]:
musical_movies['FILM_CPI'] = cpi_list

In [203]:
# Convert last performance of stage show to datetime
musical_movies['BWAY_last_performance'] = pd.to_datetime(musical_movies['BWAY_last_performance'])

In [204]:
# Create list of CPI relative to each Broadway final performance date 

test = musical_movies

bway_cpi_list = []
fail_list = []

for n, release in enumerate(test['BWAY_last_performance']):
    #print(n)
    #print(release)
    for i, cpi in enumerate(full_cpi.Date):
        if full_cpi['Date'][i] >= test['BWAY_last_performance'][n]:
            bway_cpi_list.append(full_cpi['cpi'][i])
            break
        elif full_cpi['Date'][i] > test['BWAY_last_performance'][n]:
            break
                
    #    print(i)
    #    print(full_cpi.cpi[i])

bway_cpi_list

[191.0,
 216.57299999999998,
 160.8,
 251.71400000000003,
 249.959,
 107.8,
 138.1,
 229.601,
 183.8,
 156.3,
 160.2,
 126.1,
 156.3,
 218.71099999999998,
 215.949,
 216.57299999999998,
 156.3,
 152.2,
 171.3,
 189.1,
 116.5,
 218.783,
 233.546,
 196.8,
 150.3,
 116.5,
 145.8,
 213.856,
 237.072,
 233.06900000000002,
 189.4,
 162.2,
 171.2,
 119.8,
 237.072,
 217.63099999999997,
 244.24099999999999,
 168.8,
 210.22799999999998,
 246.467,
 259.488,
 181.3,
 241.34599999999998,
 231.317,
 156.6,
 147.5,
 251.12900000000002,
 142.6,
 141.3,
 160.2,
 165.0,
 115.3,
 151.9,
 107.8,
 107.8,
 109.5,
 162.2,
 107.8,
 184.2,
 128.9,
 210.03599999999997,
 164.0,
 198.7,
 160.2,
 147.4,
 261.835,
 176.9,
 174.1,
 156.6,
 233.546,
 107.8,
 107.8,
 121.1,
 245.96400000000003,
 112.7,
 144.2,
 216.632,
 189.1,
 121.6,
 232.53099999999998,
 115.4,
 218.71099999999998,
 245.96400000000003,
 107.8,
 230.37900000000002,
 113.8,
 110.5,
 190.9,
 133.8,
 237.072,
 156.6,
 155.7,
 201.5,
 177.4,
 115.4,
 1

In [205]:
len(musical_movies['BWAY_last_performance']) - len(bway_cpi_list)

0

In [206]:
musical_movies['BWAY_CPI'] = bway_cpi_list

In [207]:
musical_movies['FILM_budget_adj'] = musical_movies['FILM_budget'] / musical_movies['FILM_CPI']
musical_movies['FILM_domestic_gross_adj'] = musical_movies['FILM_domestic_gross'] / musical_movies['FILM_CPI']
musical_movies['FILM_worldwide_gross_adj'] = musical_movies['FILM_worldwide_gross'] / musical_movies['FILM_CPI']
musical_movies['BWAY_total_weekly_gross_adj'] = musical_movies['BWAY_total_weekly_gross'] / musical_movies['BWAY_CPI']
musical_movies['BWAY_average_weekly_gross_adj'] = musical_movies['BWAY_avg_weekly_gross'] / musical_movies['BWAY_CPI']

In [208]:
musical_movies.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'title', 'FILM_runtime', 'FILM_stars',
       'FILM_directors', 'FILM_aka', 'FILM_oscars_won', 'FILM_genres',
       'FILM_budget', 'FILM_domestic_gross', 'FILM_worldwide_gross',
       'FILM_mpaa_rating', 'FILM_release_date', 'FILM_tconst',
       'FILM_primaryTitle', 'FILM_runtimeMinutes', 'FILM_genres.1',
       'BWAY_total_weekly_gross', 'BWAY_total_potential_gross',
       'BWAY_total_performances', 'BWAY_total_previews',
       'BWAY_last_performance', 'BWAY_avg_weekly_gross',
       'BWAY_avg_ticket_price', 'BWAY_avg_capacity',
       'BWAY_avg_top_ticket_price', 'BWAY_avg_seats_in_theatre',
       'FILM_release_year', 'FILM_mpaa_rating_number', 'GENRE_1', 'GENRE_2',
       'GENRE_3', 'GENRE_4', 'GENRE_5', 'GENRE_Action', 'GENRE_Adventure',
       'GENRE_Animation', 'GENRE_Biography', 'GENRE_Comedy', 'GENRE_Crime',
       'GENRE_Documentary', 'GENRE_Drama', 'GENRE_Family', 'GENRE_Fantasy',
       'GENRE_Horror', 'GENRE_Music', 'GENRE_Musical'

In [209]:
musical_movies.to_csv('data/final_dataset_10920.csv')

<h1> APPENDIX

<h2> Creation of "curiousity data set", which includes no null values for domestic gross of films

In [171]:
import pandas as pd

real_data = pd.read_csv('data/smaller_dataset_1012.csv')

In [172]:
real_data = real_data.groupby('title', group_keys=False).apply(lambda x: x.loc[x.FILM_domestic_gross.idxmax()])

real_data

,Unnamed: 0,Unnamed: 0.1,title,FILM_runtime,FILM_stars,FILM_directors,FILM_filming_locations,FILM_aka,FILM_oscars_won,FILM_genres,...,BWAY_total_performances,BWAY_total_previews,BWAY_last_performance,BWAY_avg_weekly_gross,BWAY_avg_ticket_price,BWAY_avg_capacity,BWAY_avg_top_ticket_price,BWAY_avg_seats_in_theatre,FILM_release_year,FILM_mpaa_rating_number
title,,,,,,,,,,,,,,,,,,,,,
"'night, Mother",24792,24792,"'night, Mother",96.0,"['Sissy Spacek', 'Anne Bancroft', 'Ed Berke']",['Tom Moore'],"Universal Studios - 100 Universal City Plaza, ...","'night, Mother",1.0,"['contentRating: PG-13', 'actor: [', '{', '@ty...",...,65,26,2004-10-24,1.676233e+05,52.470000,0.399467,86.250000,1078.000000,1987-01-01,3
A Bronx Tale,25469,25469,A Bronx Tale,121.0,"['Robert De Niro', 'Chazz Palminteri', 'Lillo ...",['Robert De Niro'],"4408 30th Avenue, Astoria, Queens, New York Ci...",A Bronx Tale,0.0,"['Crime', 'Drama', 'Romance']",...,700,28,2016-11-06,7.469888e+05,100.183913,0.898660,246.239130,1044.000000,1993-01-01,4
A Chorus Line,5161,5161,A Chorus Line,113.0,"['Michael Douglas', 'Terrence Mann', 'Michael ...",['Richard Attenborough'],"Helen Hayes Theater - 240 West 44th Street, Ma...",A Chorus Line,3.0,"['Drama', 'Music', 'Musical']",...,2799,18,1985-06-09,2.998957e+05,42.893915,0.645027,280.637500,1352.456338,1985-01-01,3
A Christmas Carol,6804,6804,A Christmas Carol,96.0,"['Jim Carrey', 'Gary Oldman', 'Colin Firth']",['Robert Zemeckis'],Not found,A Christmas Carol: An IMAX 3D Experience,0.0,"['Animation', 'Drama', 'Family', 'Fantasy']",...,119,19,1991-12-22,5.126635e+05,73.534444,0.874172,286.900000,1102.111111,2009-01-01,2
A Christmas Story,24643,24643,A Christmas Story,94.0,"['Peter Billingsley', 'Melinda Dillon', 'Darre...",['Bob Clark'],"232 Queen St W, Toronto, Ontario, Canada",A Christmas Story,0.0,"['Comedy', 'Family']",...,51,15,2012-11-11,1.079193e+06,94.092500,0.890312,229.250000,1509.500000,1983-01-01,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Who's Afraid of Virginia Woolf?,22530,22530,Who's Afraid of Virginia Woolf?,131.0,"['Elizabeth Taylor', 'Richard Burton', 'George...",['Mike Nichols'],"Cambridge, Massachusetts, USA",Edward Albee's Who's Afraid of Virginia Woolf?,0.0,"['contentRating: Not Rated', 'actor: [', '{', ...",...,319,25,2005-03-13,2.656996e+05,66.287347,0.628190,154.030612,947.673469,1966-01-01,0
Wonderful Town,26558,26558,Wonderful Town,92.0,"['Anchalee Saisoontorn', 'Supphasit Kansen', '...",['Aditya Assarat'],"Pakua Pak, Thailand",Cudowne miasto,0.0,"['Drama', 'Romance']",...,497,16,2003-11-09,4.589270e+05,65.619385,0.607945,95.000000,1437.000000,2008-01-01,0
Wonderland,25900,25900,Wonderland,104.0,"['Val Kilmer', 'Lisa Kudrow', 'Kate Bosworth']",['James Cox'],"California, USA",The Wonderland Murders,0.0,"['Crime', 'Drama', 'Mystery', 'Thriller']",...,33,30,2011-03-27,5.548714e+05,52.387500,0.834075,200.000000,1612.000000,2003-01-01,4


In [173]:
# Reseting index after massive cut
real_data = real_data.reset_index(drop=True)

In [174]:
# Cleans Genre category, and separates successfully scraped genres into list

import numpy as np

new_genre_list = []

for n, genre in enumerate(real_data.FILM_genres):
    if n == 0:
        genre_list = real_data.FILM_genres[n]
        genre_list = str(genre_list).replace("'", '')
        genre_list = genre_list.replace("]", '')
        genre_list = genre_list.replace("[", '')
        genre_list = genre_list.split(',')
        for i, subgenre in enumerate(genre_list):
            genre_list[i] = genre_list[i].strip()
            
        new_genre_list.append(genre_list)

        #real_data.xs(0)['FILM_genres'] = genre_list 

    else:
        genre_list = real_data.FILM_genres[n-1]
        genre_list = str(genre_list).replace("'", '')
        genre_list = genre_list.replace("]", '')
        genre_list = genre_list.replace("[", '')
        genre_list = genre_list.split(',')
        for i, subgenre in enumerate(genre_list):
            genre_list[i] = genre_list[i].strip()
            
        new_genre_list.append(genre_list)
        #real_data.xs(n-1)['FILM_genres'] = genre_list


real_data['FILM_genres'] = new_genre_list

In [175]:
# Divides genres into five lists (only first five genres accepted)

real_data['GENRE_1'] = ''
real_data['GENRE_2'] = ''
real_data['GENRE_3'] = ''
real_data['GENRE_4'] = ''
real_data['GENRE_5'] = ''

genre_options = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
                'Family', 'Fantasy', 'Film Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance',
                'Sci-Fi', 'Short', 'Sport', 'Superhero', 'Thriller', 'War', 'Western']

# Specifics the entire selection of lists of genres
for n,genre_list in enumerate(real_data.FILM_genres):
    
    # Zooms in on the individual genre in each list
    for i, genre in enumerate(genre_list):
        
        # Qualifies if that listed genre is in the IMDB list of genres
        if genre in genre_options:

            if genre == genre_list[0]:
                real_data.loc[n,'GENRE_1']=genre
            elif genre == genre_list[1]:
                real_data.loc[n,'GENRE_2']=genre
            elif genre == genre_list[2]:
                real_data.loc[n,'GENRE_3']=genre
            elif genre == genre_list[3]:
                real_data.loc[n,'GENRE_4']=genre
            elif genre == genre_list[4]:
                real_data.loc[n,'GENRE_5']=genre
            else:
                pass


In [176]:
# There was no functionally effecient way to do this - this adds the individual dummies for each of the five
# genre categories, and gives a clean dummy variable from these five genres.

dummies = pd.get_dummies(real_data[['GENRE_1','GENRE_2', 'GENRE_3','GENRE_4','GENRE_5']])

real_data['GENRE_Action'] = dummies.GENRE_1_Action
real_data['GENRE_Adventure'] = dummies.GENRE_1_Adventure + dummies.GENRE_2_Adventure
#real_data['GENRE_Adventure'] = dummies.GENRE_1_Adventure + dummies.GENRE_2_Adventure + dummies.GENRE_3_Adventure
real_data['GENRE_Animation'] = dummies.GENRE_1_Animation
real_data['GENRE_Biography'] = dummies.GENRE_1_Biography
real_data['GENRE_Comedy'] = dummies.GENRE_1_Comedy + dummies.GENRE_2_Comedy + dummies.GENRE_3_Comedy
real_data['GENRE_Crime'] = dummies.GENRE_1_Crime + dummies.GENRE_2_Crime + dummies.GENRE_3_Crime
real_data['GENRE_Documentary'] = dummies.GENRE_1_Documentary
real_data['GENRE_Drama'] = dummies.GENRE_1_Drama + dummies.GENRE_2_Drama + dummies.GENRE_3_Drama + dummies.GENRE_4_Drama
real_data['GENRE_Family'] = dummies.GENRE_1_Family + dummies.GENRE_2_Family + dummies.GENRE_3_Family + dummies.GENRE_4_Family
real_data['GENRE_Fantasy'] = dummies.GENRE_1_Fantasy + dummies.GENRE_2_Fantasy + dummies.GENRE_3_Fantasy + dummies.GENRE_5_Fantasy
real_data['GENRE_Horror'] = dummies.GENRE_1_Horror + dummies.GENRE_2_Horror
real_data['GENRE_Music'] = dummies.GENRE_2_Music + dummies.GENRE_3_Music + dummies.GENRE_4_Music
#real_data['GENRE_Music'] = dummies.GENRE_1_Music + dummies.GENRE_2_Music + dummies.GENRE_3_Music + dummies.GENRE_4_Music
real_data['GENRE_Musical'] = dummies.GENRE_1_Musical + dummies.GENRE_2_Musical + dummies.GENRE_3_Musical + dummies.GENRE_4_Musical + dummies.GENRE_5_Musical
real_data['GENRE_Mystery'] = dummies.GENRE_2_Mystery + dummies.GENRE_3_Mystery
#real_data['GENRE_Mystery'] = dummies.GENRE_1_Mystery + dummies.GENRE_2_Mystery + dummies.GENRE_3_Mystery + dummies.GENRE_4_Mystery
real_data['GENRE_Romance'] = dummies.GENRE_2_Romance + dummies.GENRE_3_Romance + dummies.GENRE_4_Romance + dummies.GENRE_5_Romance 
#musical_movies.GENRE_Sci-Fi = dummies.GENRE_2_Sci-Fi + dummies.GENRE_3_Sci-Fi + dummies.GENRE_4_Sci-Fi + dummies.GENRE_5_Sci-Fi
real_data['GENRE_Sport'] = dummies.GENRE_2_Sport + dummies.GENRE_3_Sport
real_data['GENRE_Thriller'] = dummies.GENRE_2_Thriller + dummies.GENRE_3_Thriller + dummies.GENRE_4_Thriller + dummies.GENRE_5_Thriller
real_data['GENRE_War'] = dummies.GENRE_2_War + dummies.GENRE_3_War + dummies.GENRE_5_War
#real_data['GENRE_War'] = dummies.GENRE_2_War + dummies.GENRE_3_War + dummies.GENRE_4_War + dummies.GENRE_5_War
real_data['GENRE_Western'] = dummies.GENRE_3_Western 
#real_data['GENRE_Western'] = dummies.GENRE_3_Western + dummies.GENRE_5_Western


# Reseting index after massive cut
real_data = real_data.reset_index(drop=True)

In [210]:
import fancyimpute  
from fancyimpute import IterativeImputer

from sklearn.impute import SimpleImputer 
# Getting only the float columns
continuous = real_data[['FILM_budget', 'FILM_oscars_won', 'FILM_domestic_gross', 'FILM_worldwide_gross',
                             'FILM_mpaa_rating_number', 'FILM_runtime', 'BWAY_total_weekly_gross', 'BWAY_total_potential_gross']]
MICE = IterativeImputer()
continuous = MICE.fit_transform(continuous)
continuous = pd.DataFrame(continuous, columns = ['FILM_budget', 'FILM_oscars_won', 
                                          'FILM_domestic_gross', 'FILM_worldwide_gross',
                                          'FILM_mpaa_rating_number', 
                                          'FILM_runtime', 'BWAY_total_weekly_gross', 'BWAY_total_potential_gross']).set_index(real_data.index)
continuous

,FILM_budget,FILM_oscars_won,FILM_domestic_gross,FILM_worldwide_gross,FILM_mpaa_rating_number,FILM_runtime,BWAY_total_weekly_gross,BWAY_total_potential_gross
0,1.410747e+07,1.0,441863.0,441863.0,3.0,96.0,2.011480e+06,7329105.0
1,2.200000e+07,0.0,17287898.0,17287898.0,4.0,121.0,6.872297e+07,84757024.0
2,2.700000e+07,3.0,14202899.0,14202899.0,3.0,113.0,1.064630e+08,78772513.0
3,2.000000e+08,0.0,137855863.0,325286646.0,2.0,96.0,9.227944e+06,8228171.0
4,3.300000e+06,0.0,20605209.0,20618717.0,2.0,94.0,8.633547e+06,11751311.0
...,...,...,...,...,...,...,...,...
221,7.500000e+06,0.0,28000000.0,28005068.0,0.0,131.0,1.301928e+07,29450780.0
222,1.358270e+07,0.0,4487.0,280024.0,0.0,92.0,2.983026e+07,60856677.0
223,5.000000e+06,0.0,1060512.0,2466444.0,4.0,104.0,4.438971e+06,11111216.0
224,2.000000e+07,0.0,22762571.0,22762571.0,2.0,96.0,1.712631e+07,35130630.0


In [211]:
# assign columns from MICE imputer to original df

column_list = ['FILM_budget', 'FILM_oscars_won', 'FILM_domestic_gross', 'FILM_worldwide_gross','FILM_mpaa_rating_number', 
 'FILM_runtime', 'BWAY_total_weekly_gross', 'BWAY_total_potential_gross']

for column in column_list:
    musical_movies[column] = continuous[column]
    
    

In [212]:
real_data.to_csv('data/curiousity_dataset_101020.csv')

<h3> Actors / Directors - found to be negligible in dataset

In [65]:
# cleans "Stars" list from scrape and separates it into three columns
# INCOMPLETE

musical_movies['FILM_stars1'] = ''
musical_movies['FILM_stars2'] = ''
musical_movies['FILM_stars3'] = ''

import numpy as np

musical_test = musical_movies

for n, stars in enumerate(musical_movies.FILM_stars):
    print(n)
    stars_list = musical_movies.FILM_release_date[n+1]
    print(stars_list)
    actor_list = stars_list.split(',')
    print(n)
    
    
    for i, actor in enumerate(actor_list):
        print(i, actor)
        
        '''
        musical_movies['FILM_stars'][i] = actor.replace('[', '')
        musical_movies['FILM_stars'][i] = musical_movies['FILM_stars'][i].replace(']', '')
        musical_movies['FILM_stars'][i] = musical_movies['FILM_stars'][i].replace("'", '')
        print(musical_movies['FILM_stars'][i])
        
        if i == 0:
            musical_movies['FILM_stars1'][n] = musical_movies['FILM_stars1'][i]
        elif i == 1: 
            musical_movies['FILM_stars2'][n] = musical_movies['FILM_stars2'][i]
        elif i == 2:
            musical_movies['FILM_stars3'][n] = musical_movies['FILM_stars3'][i]
        else: 
            pass
    

    for i, actors in enumerate(actor_list):
        #actor = actor.strip()
        #print(i, actors)

        if i == 0:
            musical_test['FILM_stars1'][n] = actors
            musical_test['FILM_stars1'][n] = musical_test['FILM_stars1'][n].replace("'", '')
            musical_test['FILM_stars1'][n] = musical_test['FILM_stars1'][n].replace("[", '')           
            musical_test['FILM_stars1'][n] = musical_test['FILM_stars1'][n].replace("]", '')   
            musical_test['FILM_stars1'][n] = musical_test['FILM_stars1'][n].strip()  
        elif i == 1: 
            musical_test['FILM_stars2'][n] = actors
            musical_test['FILM_stars2'][n] = musical_test['FILM_stars2'][n].replace("'", '')
            musical_test['FILM_stars2'][n] = musical_test['FILM_stars2'][n].replace("[", '')           
            musical_test['FILM_stars2'][n] = musical_test['FILM_stars2'][n].replace("]", '')   
            musical_test['FILM_stars2'][n] = musical_test['FILM_stars2'][n].strip()  
        elif i == 2:
            musical_test['FILM_stars3'][n] = actors
            musical_test['FILM_stars3'][n] = musical_test['FILM_stars3'][n].replace("'", '')
            musical_test['FILM_stars3'][n] = musical_test['FILM_stars3'][n].replace("[", '')           
            musical_test['FILM_stars3'][n] = musical_test['FILM_stars3'][n].replace("]", '')   
            musical_test['FILM_stars3'][n] = musical_test['FILM_stars3'][n].strip()
        else: 
            pass
            
    '''

0
17 March 2017
0
0 17 March 2017
1
17 March 2017
1
0 17 March 2017
2


KeyError: 3

In [ ]:
# separating values for stars from values without stars, to be concatenated below

stars = musical_test[musical_test['FILM_stars1'] != "['Rosemarie Sonora'"]
no_stars = musical_test[musical_test['FILM_stars1'] == "['Rosemarie Sonora'"]

no_stars[['FILM_stars1', 'FILM_stars2', 'FILM_stars3']] = 'none'


In [ ]:
# FINAL assignment

musical_movies = pd.concat([stars,no_stars])

In [ ]:
# cleans "Directors" list from scrape and separates it into three columns

musical_movies['FILM_directors1'] = ''
musical_movies['FILM_directors2'] = ''
musical_movies['FILM_directors3'] = ''

import numpy as np

for n, directors in enumerate(musical_movies.FILM_directors):
    director_list = musical_movies.FILM_directors[n]
    director_list = director_list.split(',')
    print(n)
    
    for i, director in enumerate(director_list):
        #director = director.strip()
        print(i, director)
        musical_movies['FILM_directors'][i] = director.replace('[', '')
        musical_movies['FILM_directors'][i] = musical_movies['FILM_directors'][i].replace(']', '')
        musical_movies['FILM_directors'][i] = musical_movies['FILM_directors'][i].replace("'", '')
        print(musical_movies['FILM_directors'][i])
        
        if i == 0:
            musical_movies['FILM_directors1'][n] = musical_movies['FILM_directors'][i]
        elif i == 1: 
            musical_movies['FILM_directors2'][n] = musical_movies['FILM_directors'][i]
        elif i == 2:
            musical_movies['FILM_directors3'][n] = musical_movies['FILM_directors'][i]
        else: 
            pass

In [ ]:
musical_movies['FILM_genres1'] = ''
musical_movies['FILM_genres2'] = ''
musical_movies['FILM_genres3'] = ''
musical_movies['FILM_genres4'] = ''
musical_movies['FILM_genres5'] = ''

import numpy as np

for n, genres in enumerate(musical_movies.FILM_genres):
    genre_list = musical_movies.FILM_genres[n+1]
    genre_list = genre_list.split(',')
    print(genre_list)

    for i, genre in enumerate(genre_list):
        #director = director.strip()
        print(i, genre)
        musical_movies['FILM_genres'][i] = genre.replace('[', '')
        musical_movies['FILM_genres'][i] = musical_movies['FILM_genres'][i].replace(']', '')
        musical_movies['FILM_genres'][i] = musical_movies['FILM_genres'][i].replace("'", '')
        print(musical_movies['FILM_genres'][i])
        
        if i == 0:
            musical_movies['FILM_genres1'][n] = musical_movies['FILM_genres1'][i]
        elif i == 1: 
            musical_movies['FILM_genres2'][n] = musical_movies['FILM_genres2'][i]
        elif i == 2:
            musical_movies['FILM_genres3'][n] = musical_movies['FILM_genres3'][i]
        elif i == 3:
            musical_movies['FILM_genres4'][n] = musical_movies['FILM_genres4'][i]
        elif i == 4:
            musical_movies['FILM_genres5'][n] = musical_movies['FILM_genres5'][i]           
        else: 
            pass